# Data chunking

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)
### Contributors
- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)
- Anne Fouilloux, University of Oslo (Norway), [@annefou](https://github.com/annefou)



<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>Why do chunking matter?</li>
        <li>How can I read datasets by chunks to optimize memory usage?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about chunking</li>
        <li>Learn about zarr </li>
        <li>Use kerchunk to consolidate chunk metadata and prepare single ensemble datasets for parallel computing</li>
    </ul>
</div>

## Context

When dealing with large data files or collections, it's often impossible to load all the data you want to analyze into a single computer memory at once.  This is a situation where the Pangeo ecosystem can help you a lot. Xarray offers the possibility to work lazily on data __chunks__, which means pieces of an entire dataset.  By taking a data as 'chunk' we can process our data peace by peace on a single computer using local dask cluster, or distributed computing cluster.  How we will process these 'chunks' in a parallel enviroment will be discussed in [dask_introduction](./dask_introduction.ipynb)  The idia of __chunk__  will be explained here.

When we process our data peace by peace, we would like to have our original data or processed result also save in 'chunks'. [Zarr](https://zarr.readthedocs.io/en/stable/) is a majour way used in pangeo ecosystem to save our xarray dataset in __chunk__.

Zarr is not the only file format which use __chunk__. We will also be using [kerchunk library](https://fsspec.github.io/kerchunk/) in this notebook to build a virtual __chunked__ dataset based on NetCDF, and show how it optimizes the access and analysis of large datasets.

The analysis is very similar to what we have done in previous episodes, however we will use data on a global coverage and not only on a small geographical area (e.g. Lombardia).

### Data

In this episode, we will be using Global Long Term Statistics (1999-2019) products provided by the [Copernicus Global Land Service](https://land.copernicus.eu/global/index.html) and access them through [S3-comptabile storage](https://en.wikipedia.org/wiki/Amazon_S3) ([OpenStack Object Storage "Swift"](https://wiki.openstack.org/wiki/Swift)) with a data catalog we have created and made publicly available.

## Setup

This episode uses the following Python packages:

- fsspec
- s3fs
- pooch
- xarray
- dask
- netcdf4
- h5netcdf
- hvplot
- kerchunk
- pprint
- json
- geopandas
- matplotlib

Please install these packages if not already available in your Python environment (see [Setup page](https://pangeo-data.github.io/foss4g-2022/before/setup.html)).

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## Global LTS

In the previous episode, we used Long Term tastitics time-series for the region of Lombardy e.g. a very small area. Now we would like to use the original dataset that has a global coverage. Let us first open a single file (for January 1999-2019) to understand how much larger the global dataset is.

In [8]:
import fsspec
import s3fs
import xarray as xr

In [9]:
fs = s3fs.S3FileSystem(anon=True,
      client_kwargs={
         'endpoint_url': 'https://object-store.cloud.muni.cz'
      })

As shown in the [Data discovery](./data_discovery.ipynb) chapter, when we have several files to read at once, we need to use Xarray `open_mfdataset` instead of `open_dataset`. It takes a list in input, and not a single element. We can also use `open_mfdataset` with one file as done below.

Using `open_mfdataset` automatically switch from Numpy Arrays to Dask Arrays as the data structure used by Xarray. The same happens when specifying chunks in `open_dataset` as you'll see below.

In [10]:
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'

In [11]:
%%time
LTS = xr.open_mfdataset([fs.open(s3path)])
LTS

CPU times: user 116 ms, sys: 9.6 ms, total: 125 ms
Wall time: 1.64 s


<xarray.Dataset>
Dimensions:  (lon: 40320, lat: 15680)
Coordinates:
  * lon      (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * lat      (lat) float64 80.0 79.99 79.98 79.97 ... -59.97 -59.98 -59.99
Data variables:
    crs      |S1 ...
    min      (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
    median   (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
    max      (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
    mean     (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
    stdev    (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
    nobs     (lat, lon) float32 dask.array<chunksize=(15680, 40320), meta=np.ndarray>
Attributes: (12/19)
    Conventions:          CF-1.6
    parent_identifier:    urn:cgls:global:ndvi_stats_all
    identifier:           urn:cgls:global:ndvi_stats_all:NDVI-LTS_1999-2019-1...
    long_name:            Normalized Difference Vegetation Index
    title:                Normalized Difference Vegetation Index: Long Term S...
    product_version:      V3.0.1
    ...                   ...
    source:               Derived from EO satellite imagery
    processing_mode:      Offline
    references:           https://land.copernicus.eu/global/products/ndvi
    copyright:            Copernicus Service information 2021
    archive_facility:     VITO
    history:              2021-03-01 - Processing line NDVI LTS

## What is __chunk__

You may have missed it but if you look carefully to `LTS`, each Data Variable is a `dask.array` with a chunsize of `(15680, 40320)`. So basically accessing one data variable would load arrays of dimensions `(15680, 40320)` into the memory.  You can have the information by clicking the icon indicated in the image below

![Dask.array](../figures/datasize.png)

When you open a netCDF file, by default, the chunks correspond to the entire size of the dataset originated to one file. When you need to analyze a considerable number of large files, the memory may not be sufficient anymore. This is where understanding chunking comes into play.



In [ ]:
LTS.sel(lat=slice(80.,70.),lon=slice(70.,90))

When selecting you may have the feeling that the chunk sizes changes. In fact Xarray will still have to fetch the entire initial chunk to perform the selection on any of the Data variables. Again it will not be very optimal (your Python Jupyter kernel may crash too!) with large numbers of files and large files. 

## Exploiting native chunking for reading datasets

Many data formats (for instance, [netCDF4](https://unidata.github.io/netcdf4-python/), [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) or [geoTIFF](https://en.wikipedia.org/wiki/GeoTIFF)) chunk natively the datasets (this is done at the creation of the datasets). These native chunks can be retrieved and used when opening and accessing the files. This will allow significantly reducing the amount of memory when analyzing Data Variables (only the needed chunks will be transferred, and if all the data have to be accessed it can be serializable e.g. chunks are processed one after the other).

Xarray can use this native chunking for you automatically by using the `chunks` argument when opening a dataset.

In [ ]:
%%time
LTS = xr.open_dataset(fs.open(s3path), chunks='auto')
LTS

Here, you can see by opening the detail of a data variable that Xarray choose to use chunks of (3136, 8064) shape, which means about 100MB chunks which is generally a good volume.

### Extract chunk information

We extract chunk information from each file using `kerchunk.hdf`.
Let's start with a single file.

Kerchunk will help a lot when we'll want to read all of the NetCDF files from our Dataset by building a consolidated metadata files for all the chunks from all of our files.

In [ ]:
import kerchunk.hdf

We use `kerchunk.hdf` because our files are written in `netCDF4`  format which is based on HDF5 and `SingleHdf5ToZarr` to translate the metadata of one HDF5 file into Zarr metadata format. The parameter `inline_threshold` is an *optimization* and tells `SingleHdf5ToZarr` to include chunks smaller than this value directly in the output. 

In [ ]:
remote_filename = 'https://object-store.cloud.muni.cz/swift/v1/foss4g-data/CGLS_LTS_1999_2019/c_gls_NDVI-LTS_1999-2019-1221_GLOBE_VGT-PROBAV_V3.0.1.nc'
with fsspec.open(remote_filename) as inf:
    h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, remote_filename, inline_threshold=100)
    chunk_info = h5chunks.translate()

Let's have a look at `chunk_info`. It is a Python dictionary so we can use `pprint` to print it nicely.

Content is a bit complicated, but it's only metadata in Zarr format indicating what's in the original file, and where the chunks of the file are located (bytes offset).

In [ ]:
from pprint import pprint
pprint(chunk_info)

After we have collected information on the native chunks in the original data file and consolidated our Zarr metadata, we can open the files using `zarr` and pass this chunk information into a storage option. We also need to pass `"consolidated": False` because the original dataset does not contain any `zarr` consolidating metadata.

In [ ]:
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": chunk_info,
        },
        "consolidated": False
    }
)
LTS

As you can notice above, all the Data Variables have several chunks. The native chunking of the NetCDF file.

### Combine all LTS files into one kerchunked single ensemble dataset

Now we will combine all the files into one kerchunked consolidated dataset, and try to open it as a xarray dataset. 

Let us first collect the chunk information for each file.

In [ ]:
fs.ls('foss4g-data/CGLS_LTS_1999_2019')

In [ ]:
from datetime import datetime

In [ ]:
s3path = 's3://foss4g-data/CGLS_LTS_1999_2019/c_gls_*.nc'
chunk_info_list = []
time_list = []

for file in fs.glob(s3path):
    url = 'https://object-store.cloud.muni.cz/swift/v1/' + file
    t = datetime.strptime(file.split('/')[-1].split('_')[3].replace('1999-', ''), "%Y-%m%d")
    time_list.append(t)
    with fsspec.open(url) as inf:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, url, inline_threshold=100)
        chunk_info_list.append(h5chunks.translate())

This time we use `MultiZarrToZarr` to combine multiple kerchunked datasets into a single logical aggregated dataset. Like when opening multiple files with Xarray `open_mfdataset`, we need to tell `MultiZarrToZarr` how to concatenate all the files. There is no time dimension in the original dataset, but one file corresponds to one date (average over the period 1999-2019 for a given 10-day period e.g. January 01, January 11, January 21, etc.).

In [ ]:
from kerchunk.combine import MultiZarrToZarr
mzz = MultiZarrToZarr(
    chunk_info_list,
    coo_map={'INDEX': 'INDEX'},
    identical_dims=['crs'],
    concat_dims=["INDEX"],
)

out = mzz.translate()

Then, we can open the complete dataset using our consolidated Zarr metadata.

In [ ]:
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": out,
        },
        "consolidated": False
    }
)
LTS

We can save the consolidated metadata for our dataset in a file, and reuse it later to access the dataset.

In [ ]:
import json

In [ ]:
jsonfile='c_gls_NDVI-LTS_1999-2019.json'
with open(jsonfile, mode='w') as f :
    json.dump(out, f)

We can then load data from this catalog.

In [ ]:
import xarray as xr
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":'./c_gls_NDVI-LTS_1999-2019.json',
        },
        "consolidated": False
    }
)
LTS

We can also postprocess LTS to get the dates properly set.

In [ ]:
LTS = LTS.rename({'INDEX': 'time'}).assign_coords(time=time_list)
LTS

In [ ]:
import geopandas as gpd

In [ ]:
try:
    GAUL = gpd.read_file('Italy.geojson')
except:
    GAUL = gpd.read_file('zip+https://mars.jrc.ec.europa.eu/asap/files/gaul1_asap.zip') 

In [ ]:
AOI_name = 'Lombardia'
AOI = GAUL[GAUL.name1 == AOI_name]
AOI_poly = AOI.geometry
AOI_poly

In [ ]:
LTS = LTS.sel(lat=slice(46.5,44.5), lon=slice(8.5,11.5))
LTS = LTS.rio.write_crs(4326)

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

In [ ]:
LTS

In [ ]:
%%time
LTS.compute()

## Visualize LTS statistics

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(1, figsize=[15,5])

# Fix extent
minval = 0.0
maxval = 0.9

itime=0 # plot the first date

# Plot 1 for min subplot argument (nrows, ncols, nplot)
# here 1 row, 2 columns and 1st plot
ax1 = plt.subplot(1, 2, 1)
LTS.isel(time=itime)['min'].plot(ax=ax1)
# Plot 2 for max
# 2nd plot 
ax2 = plt.subplot(1, 2, 2)
LTS.isel(time=itime)['max'].plot(ax=ax2)

# Title for both plots
fig.suptitle('LTS NDVI statistics (Minimum and Maximum)', fontsize=20)

The catalog (json file we created) can be shared on the cloud (or github, etc.) and anyone can load it from there too.
This approach allows anyone to easily access LTS data and select the Area of Interest for their own study.

## Conclusion

Understanding chunking is key to optimize your data analysis. In this episode we learned how to optimize the memory by exploiting native chunks from netCDF4 data files and instructing Xarray to access data per chunk. However, computations can be very slow and to optimize the computational part we need to parallelize our data analysis. This is what you will learn in the next episode with Dask.

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Chunking and compression</li>
        <li>kerchunk</li>
    </ul>
</div>